In [1]:
import pandas as pd

In [10]:
alibaba_batch_task_df = pd.read_csv('data/alibaba/batch_task_chunk_10000.csv')
alibaba_batch_task_df.head()

,task_name,instance_num,job_name,task_type,status,start_time,end_time,plan_cpu,plan_mem
0,M1,1.0,j_1,1,Terminated,419912,419912,100.0,0.20
1,R2_1,1.0,j_2,1,Terminated,87076,87086,50.0,0.20
2,M1,1.0,j_2,1,Terminated,87076,87083,50.0,0.20
3,R6_3,371.0,j_3,1,Terminated,157297,157325,100.0,0.49
4,J4_2_3,1111.0,j_3,1,Terminated,157329,157376,100.0,0.59


In [11]:
alibaba_batch_task_df = alibaba_batch_task_df[alibaba_batch_task_df['status'] == 'Terminated']
alibaba_batch_task_df.head()

,task_name,instance_num,job_name,task_type,status,start_time,end_time,plan_cpu,plan_mem
0,M1,1.0,j_1,1,Terminated,419912,419912,100.0,0.20
1,R2_1,1.0,j_2,1,Terminated,87076,87086,50.0,0.20
2,M1,1.0,j_2,1,Terminated,87076,87083,50.0,0.20
3,R6_3,371.0,j_3,1,Terminated,157297,157325,100.0,0.49
4,J4_2_3,1111.0,j_3,1,Terminated,157329,157376,100.0,0.59


In [13]:
alibaba_batch_task_df = alibaba_batch_task_df.sort_values(by='start_time', ascending=True)
alibaba_batch_task_df.reset_index(drop=True, inplace=True)
alibaba_batch_task_df.head()

,task_name,instance_num,job_name,task_type,status,start_time,end_time,plan_cpu,plan_mem
0,M2,40.0,j_1661,1,Terminated,86207,86485,50.0,0.39
1,J7_1_2_6,1.0,j_1661,1,Terminated,86207,86622,50.0,0.20
2,M3,1.0,j_1661,1,Terminated,86207,86210,50.0,0.20
3,R4_3,1.0,j_1661,1,Terminated,86207,86212,50.0,0.20
4,M5_4,17.0,j_1661,1,Terminated,86207,86300,50.0,0.39


In [9]:
status_categories = alibaba_batch_task_df['status'].unique()

# Display all status categories (haz el mergetask ser -1 o lo que quieras y las otras unit_task o lo quequieras)
print(status_categories)

['Waiting' 'Terminated' 'Running' 'Failed']
